<a href="https://colab.research.google.com/github/BradySark/Site/blob/main/etl_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Demonstration of ETL procedure. I will use Amazon data "scraped"/**Extracted** using the BeautifulSoup4 library, **Transfer** it into a dataframe using the Pandaslibrary and save it as a csv file. Finally, I will create a SQLite3 database and **Load** the extracted data into it.

STEP 1: Loading required libraries

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

STEP 2: Python code to **Extract** Web Data from Amazon. Top 30 Best Selling Books 

In [ ]:
# Scraping data from web page for Amazon-Best Sellers
url = 'https://www.amazon.com/gp/bestsellers/books/11892'
request = Request(url, headers={'User-agent': 'Mozilla/5.0'})
html = urlopen(request)

soup = BeautifulSoup(html, 'html.parser')

books = soup.find_all('div', id="gridItemRoot")
#print(books)
bookdata=[]

for book in books:
    rank = book.find('span', class_="zg-bdg-text").get_text().replace('#', '')
    #print(rank)
    title = book.find('div', class_="_cDEzb_p13n-sc-css-line-clamp-1_1Fn1y").get_text(strip=True)
    #print(f"Title: {title}")
    author = book.find('div', class_="a-row a-size-small").get_text(strip=True)
    #print(f"Author: {author}")
    bookdata.append([rank, title, author])

#print(bookdata)
booksdf = pd.DataFrame(bookdata, columns = ['Ranking', 'Title', 'Author'])
booksdf


,Ranking,Title,Author
0,1,On Writing: A Memoir of the Craft,Stephen King
1,2,Julia Cameron,Julia Cameron
2,3,The Emotion Thesaurus: A Writer's Guide to Cha...,Becca Puglisi
3,4,Writing Down the Bones: Freeing the Writer Within,Natalie Goldberg
4,5,A Poetry Handbook,Mary Oliver
5,6,Maps and Legends: Reading and Writing Along th...,Michael Chabon
6,7,The Only World Building Workbook You'll Ever N...,T.M. Holladay
7,8,The Art and Business of Online Writing: How to...,Nicolas Cole
8,9,The Secrets to Creating Character Arcs: A Fict...,John S. Warner
9,10,The Emotional Wound Thesaurus: A Writer's Guid...,Becca Puglisi


STEP 3: Found an issue. Two authors names, Julia Cameron and Jenna Moreci, ended up in their "Titles". Using Pandas to clean the data.

In [ ]:
booksdf['Title'] = booksdf['Title'].replace(['Julia Cameron'], 'Write For Life')
booksdf['Title'] = booksdf['Title'].replace(['Jenna Moreci'], 'Shut Up and Write the Book')
booksdf


,Ranking,Title,Author
0,1,On Writing: A Memoir of the Craft,Stephen King
1,2,Write For Life,Julia Cameron
2,3,The Emotion Thesaurus: A Writer's Guide to Cha...,Becca Puglisi
3,4,Writing Down the Bones: Freeing the Writer Within,Natalie Goldberg
4,5,A Poetry Handbook,Mary Oliver
5,6,Maps and Legends: Reading and Writing Along th...,Michael Chabon
6,7,The Only World Building Workbook You'll Ever N...,T.M. Holladay
7,8,The Art and Business of Online Writing: How to...,Nicolas Cole
8,9,The Secrets to Creating Character Arcs: A Fict...,John S. Warner
9,10,The Emotional Wound Thesaurus: A Writer's Guid...,Becca Puglisi


STEP 4: With the data "cleaned", it is ready to be saved as a csv file.

In [ ]:
booksdf.to_csv('best_sellerCLEAN.csv', index=False)
cleandf = pd.read_csv('best_sellerCLEAN.csv')
cleandf

,Ranking,Title,Author
0,1,On Writing: A Memoir of the Craft,Stephen King
1,2,Write For Life,Julia Cameron
2,3,The Emotion Thesaurus: A Writer's Guide to Cha...,Becca Puglisi
3,4,Writing Down the Bones: Freeing the Writer Within,Natalie Goldberg
4,5,A Poetry Handbook,Mary Oliver
5,6,Maps and Legends: Reading and Writing Along th...,Michael Chabon
6,7,The Only World Building Workbook You'll Ever N...,T.M. Holladay
7,8,The Art and Business of Online Writing: How to...,Nicolas Cole
8,9,The Secrets to Creating Character Arcs: A Fict...,John S. Warner
9,10,The Emotional Wound Thesaurus: A Writer's Guid...,Becca Puglisi


Ready to build a SQLite3 database, and load the data. I checked to make sure it loaded correctly and is able to be queried and manipulated as needed.

In [ ]:
conn = sqlite3.connect('best_seller.db')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS Books')
cur.execute('''CREATE TABLE Books (Ranking int, Title text, Author text)''')

BSBooks = pd.read_csv('best_sellerCLEAN.csv')
BSBooks.to_sql('Books', conn, if_exists='append', index = False)

 
data=cur.execute('''SELECT * FROM Books''')
for row in data:
    print(row)

print('Voila!') 

conn.close()

(1, 'On Writing: A Memoir of the Craft', 'Stephen King')
(2, 'Write For Life', 'Julia Cameron')
(3, "The Emotion Thesaurus: A Writer's Guide to Character Expression (Second Edition) (Writers Helping Writers Series)", 'Becca Puglisi')
(4, 'Writing Down the Bones: Freeing the Writer Within', 'Natalie Goldberg')
(5, 'A Poetry Handbook', 'Mary Oliver')
(6, 'Maps and Legends: Reading and Writing Along the Borderlands', 'Michael Chabon')
(7, "The Only World Building Workbook You'll Ever Need: Your New Setting Bible (Series Bibles for Writers)", 'T.M. Holladay')
(8, 'The Art and Business of Online Writing: How to Beat the Game of Capturing and Keeping Attention', 'Nicolas Cole')
(9, "The Secrets to Creating Character Arcs: A Fiction Writer's Guide to Masterful Character Creation", 'John S. Warner')
(10, "The Emotional Wound Thesaurus: A Writer's Guide to Psychological Trauma (Writers Helping Writers Series)", 'Becca Puglisi')
(11, 'Writer´s Workbook: A Personal Planner with Tips, Checklists a